In [ ]:
import pandas as pd
from random import randint
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
'''
Analisys via bechmarks
'''

ncg_file_name = "NCG6_tsgoncogene.tsv"
cosmic_file_name = "CGC_COSMIC_Census_allMon Apr 15 17_32_47 2019.tsv"

ncg_df = pd.read_csv(ncg_file_name, sep='\t', usecols=["symbol"])
cosmic_df = pd.read_csv(cosmic_file_name, sep='\t', usecols=["Gene Symbol"])

ncg_genes = list(ncg_df["symbol"])
cosmic_genes = list(cosmic_df["Gene Symbol"])

benchmark_genes = set(ncg_genes) | set(cosmic_genes)
print("Number of genes in benchmark: {}".format(len(benchmark_genes)))

In [ ]:
'''
Input: a GeNWeMME file name with result
Output: a list with priorited genes
'''

def get_gene_list(results_file_name):
    results_df = pd.read_csv(results_file_name, sep = '\t', usecols=["component"])

    genes_result = []
    components = list(results_df["component"])
    for index, component in enumerate(components):
        component = component[1:-1] # removing brackets
        component = component.split(", ") # spliting the genes
        for gene in component:
            if gene not in genes_result:
                genes_result.append(gene)

    return genes_result

In [ ]:
'''

Output: precision
'''

def benchmark_analysis(genes_result, benchmark_genes, N):
    into_benchmark = []
    out_benchmark = []
    genes_result = genes_result[0:N]

    for gene in genes_result:
        if gene in benchmark_genes:
            into_benchmark.append(gene)
        else:
            out_benchmark.append(gene)
    
    precision = len(into_benchmark) / (len(into_benchmark) + len(out_benchmark))
    #print(into_benchmark)
    #print(out_benchmark)
    return precision
    

In [ ]:
'''
compare results with random
'''
def random_benchmark_analysis_maf(maf_file_name, benchmark_genes, N):
    maf = pd.read_csv(maf_file_name, sep = '\t', usecols=["Hugo_Symbol"])
    
    precisions = []

    for k in range(100): # 100 execuções aleatorias
        all_genes_maf = list(set(maf["Hugo_Symbol"]))
        genes_result_random = []
        for i in range(0, N):
            index_random = randint(0, len(all_genes_maf) - 1)
            genes_result_random.append(all_genes_maf[index_random])
            del(all_genes_maf[index_random])

        into_benchmark = []
        out_benchmark = []

        for gene in genes_result_random:
            if gene in benchmark_genes:
                into_benchmark.append(gene)
            else:
                out_benchmark.append(gene)

        precision = len(into_benchmark) / (len(into_benchmark) + len(out_benchmark))
        precisions.append(precision)

    return (sum(precisions) / len(precisions))



In [ ]:
# Testing
K = 5
top_N = 30
N_list = [N for N in range(K, top_N + 1)]

results_file_name = "../outputExamples/output_gbm_MEMo_5genes/gbm_MEMo_5genes.ranking"
maf_file_name = "../inputExamples/gbm_MEMo/data_mutations.maf"
gene_results = get_gene_list(results_file_name)
precision_gbm = []
precision_gbm_random = []
for N in range(K, top_N + 1):
    precision = benchmark_analysis(gene_results, benchmark_genes, N)
    precision_gbm.append(precision)
    precision_random = random_benchmark_analysis_maf(maf_file_name, benchmark_genes, N)
    precision_gbm_random.append(precision_random)

results_file_name = "../outputExamples/output_ovarian_MEMo_5genes/ovarian_MEMo_5genes.ranking"
maf_file_name = "../inputExamples/ovarian_MEMo/ovarian_mutations.maf"
gene_results = get_gene_list(results_file_name)
precision_ovarian = []
precision_ovarian_random = []
for N in range(K, top_N + 1):
    precision = benchmark_analysis(gene_results, benchmark_genes, N)
    precision_ovarian.append(precision)
    precision_random = random_benchmark_analysis_maf(maf_file_name, benchmark_genes, N)
    precision_ovarian_random.append(precision_random)
    
results_file_name = "../outputExamples/output_prostate_5genes/prostate_5genes.ranking"
maf_file_name = "../inputExamples/prostate/prostate.maf"
gene_results = get_gene_list(results_file_name)
precision_prostate = []
precision_prostate_random = []
for N in range(K, top_N + 1):
    precision = benchmark_analysis(gene_results, benchmark_genes, N)
    precision_prostate.append(precision)
    precision_random = random_benchmark_analysis_maf(maf_file_name, benchmark_genes, N)
    precision_prostate_random.append(precision_random)
    
results_file_name = "../outputExamples/output_lung_Ding2008_5genes/lung_Ding2008_5genes.ranking"
maf_file_name = "../inputExamples/lung_Ding2008/lung_Ding2008.maf"
gene_results = get_gene_list(results_file_name)
precision_lung = []
precision_lung_random = []
for N in range(K, top_N + 1):
    precision = benchmark_analysis(gene_results, benchmark_genes, N)
    precision_lung.append(precision)
    precision_random = random_benchmark_analysis_maf(maf_file_name, benchmark_genes, N)
    precision_lung_random.append(precision_random)

# print(sum(precision_gbm)/len(precision_gbm))
# print(sum(precision_ovarian)/len(precision_ovarian))
# print(sum(precision_prostate)/len(precision_prostate))
# print(sum(precision_lung)/len(precision_lung))


f, ((gbm, ovarian), (prostate, lung)) = plt.subplots(2, 2)
f.set_size_inches(12, 6)
f.subplots_adjust(hspace=0.4)

gbm.set_title("GBM")
gbm.plot(N_list, precision_gbm, '.-', label="GeNWeMME")
gbm.plot(N_list, precision_gbm_random, '|-', label="Random")
gbm.set_ylabel('Precision')
gbm.legend()
gbm.set_ylim(0.0, 1.1);

ovarian.set_title("Ovarian")
ovarian.plot(N_list, precision_ovarian, '.-')
ovarian.plot(N_list, precision_ovarian_random, '|-')
ovarian.set_ylim(0.0, 1.1);

prostate.set_title("Prostate")
prostate.plot(N_list, precision_prostate, '.-')
prostate.plot(N_list, precision_prostate_random, '|-')
prostate.set_ylabel('Precision')
prostate.set_xlabel('Top N genes')
prostate.set_ylim(0.0, 1.1);

lung.set_title("Lung")
lung.plot(N_list, precision_lung, '.-')
lung.plot(N_list, precision_lung_random, '|-')
lung.set_xlabel('Top N genes')
lung.set_ylim(0.0, 1.1);

plt.show()

In [ ]:
# Testing
from matplotlib.ticker import FormatStrFormatter
def get_mutation_matrix_from_maf(maf_file_name):
    maf = pd.read_csv(maf_file_name, sep="\t", usecols=["Hugo_Symbol", "Tumor_Sample_Barcode", "Variant_Classification"])
    mut = pd.crosstab(maf.Tumor_Sample_Barcode, maf.Hugo_Symbol).clip_upper(1)
    return mut

N = 10

f, ((gbm, ovarian), (prostate, lung)) = plt.subplots(2, 2)
f.set_size_inches(12, 6)
f.subplots_adjust(hspace=0.4)

freqs = []
cancer_type = []

results_file_name = "../outputExamples/output_gbm_MEMo_5genes/gbm_MEMo_5genes.ranking"
maf_file_name = "../inputExamples/gbm_MEMo/data_mutations.maf"
bmm = get_mutation_matrix_from_maf(maf_file_name)
sample_count, gene_count = bmm.shape
gene_results = get_gene_list(results_file_name)
gene_results = gene_results[0:N]
for gene in gene_results:
    q = bmm[gene].sum()
    freq = q / sample_count
    freqs.append(freq)
    cancer_type.append("Genes prioritized")
print("GBM:", gene_results)
    
genes = list(bmm.columns)
for gene in genes:
    q = bmm[gene].sum()
    freq = q / sample_count
    freqs.append(freq)
    cancer_type.append("All genes")

gbm.set_title("GBM")
gbm.set_ylabel("Frequency")
sns.stripplot(cancer_type, freqs, jitter=0.2, ax=gbm)
    
freqs = []
cancer_type = []

results_file_name = "../outputExamples/output_ovarian_MEMo_5genes/ovarian_MEMo_5genes.ranking"
maf_file_name = "../inputExamples/ovarian_MEMo/ovarian_mutations.maf"
bmm = get_mutation_matrix_from_maf(maf_file_name)
sample_count, gene_count = bmm.shape
gene_results = get_gene_list(results_file_name)
gene_results = gene_results[0:N]
for gene in gene_results:
    q = bmm[gene].sum()
    freq = q / sample_count
    freqs.append(freq)
    cancer_type.append("Genes prioritized")
print("Ovarian:", gene_results)
    
genes = list(bmm.columns)
for gene in genes:
    q = bmm[gene].sum()
    freq = q / sample_count
    freqs.append(freq)
    cancer_type.append("All genes")

ovarian.set_title("Ovarian")
sns.stripplot(cancer_type, freqs, jitter=0.2, ax=ovarian)

freqs = []
cancer_type = []

results_file_name = "../outputExamples/output_prostate_5genes/prostate_5genes.ranking"
maf_file_name = "../inputExamples/prostate/prostate.maf"
bmm = get_mutation_matrix_from_maf(maf_file_name)
sample_count, gene_count = bmm.shape
gene_results = get_gene_list(results_file_name)
gene_results = gene_results[0:N]
for gene in gene_results:
    q = bmm[gene].sum()
    freq = q / sample_count
    freqs.append(freq)
    cancer_type.append("Genes prioritized")
print("Prostate:", gene_results)
    
genes = list(bmm.columns)
for gene in genes:
    q = bmm[gene].sum()
    freq = q / sample_count
    freqs.append(freq)
    cancer_type.append("All genes")

prostate.set_title("Prostate")
prostate.set_ylabel("Frequency")
#prostate.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
#prostate.set_ylim(-0.01, 0.17)
sns.stripplot(cancer_type, freqs, jitter=0.2, ax=prostate)
    
freqs = []
cancer_type = []

results_file_name = "../outputExamples/output_lung_Ding2008_5genes/lung_Ding2008_5genes.ranking"
maf_file_name = "../inputExamples/lung_Ding2008/lung_Ding2008.maf"
bmm = get_mutation_matrix_from_maf(maf_file_name)
sample_count, gene_count = bmm.shape
gene_results = get_gene_list(results_file_name)
gene_results = gene_results[0:N]
for gene in gene_results:
    q = bmm[gene].sum()
    freq = q / sample_count
    freqs.append(freq)
    cancer_type.append("Genes prioritized")
print("Lung:", gene_results)
    
genes = list(bmm.columns)
for gene in genes:
    q = bmm[gene].sum()
    freq = q / sample_count
    freqs.append(freq)
    cancer_type.append("All genes")

lung.set_title("Lung")
sns.stripplot(cancer_type, freqs, jitter=0.2, ax=lung)


plt.show()